# Master Discogs Database

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-12-16 23:00:04.261862


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

In [3]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Dec 16, 2019 23:00:05 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-Metadata.p  	0 5952
/Volumes/Music/Discog/artists-db/metadata/1-Metadata.p  	1 12135
/Volumes/Music/Discog/artists-db/metadata/10-Metadata.p  	2 18156
/Volumes/Music/Discog/artists-db/metadata/11-Metadata.p  	3 24245
/Volumes/Music/Discog/artists-db/metadata/12-Metadata.p  	4 30388
/Volumes/Music/Discog/artists-db/metadata/13-Metadata.p  	5 36553
/Volumes/Music/Discog/artists-db/metadata/14-Metadata.p  	6 42532
/Volumes/Music/Discog/artists-db/metadata/15-Metadata.p  	7 48655
/Volumes/Music/Discog/artists-db/metadata/16-Metadata.p  	8 54707
/Volumes/Music/Discog/artists-db/metadata/17-Metadata.p  	9 60835
/Volumes/Music/Discog/artists-db/metadata/18-Metadata.p  	10 66965
/Volumes/Music/Discog/artists-db/metadata/19-Metadata.p  	11 72979
/Volumes/Music/Discog/artists-db/metadata/2-Metadata.p 

## Artist ID --> Albums

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Dec 16, 2019 23:01:18 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-MediaMetadata.p 	5952      3271      16797     
/Volumes/Music/Discog/artists-db/metadata/1-MediaMetadata.p 	12135     6600      34886     
/Volumes/Music/Discog/artists-db/metadata/10-MediaMetadata.p 	18156     9889      52325     
/Volumes/Music/Discog/artists-db/metadata/11-MediaMetadata.p 	24245     13179     69785     
/Volumes/Music/Discog/artists-db/metadata/12-MediaMetadata.p 	30388     16531     87328     
/Volumes/Music/Discog/artists-db/metadata/13-MediaMetadata.p 	36553     19837     104886    
/Volumes/Music/Discog/artists-db/metadata/14-MediaMetadata.p 	42532     23098     121890    
/Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p 	48655     26379     139851    
/Volumes/Music/Discog/artists-db/metadata/16-MediaMetadata.p 	54707     29657     157381    
/Volumes/Mu

/Volumes/Music/Discog/artists-db/metadata/87-MediaMetadata.p 	526415    284386    1523658   
/Volumes/Music/Discog/artists-db/metadata/88-MediaMetadata.p 	532311    287530    1540903   
/Volumes/Music/Discog/artists-db/metadata/89-MediaMetadata.p 	538424    290824    1558466   
/Volumes/Music/Discog/artists-db/metadata/9-MediaMetadata.p 	544360    294008    1575810   
/Volumes/Music/Discog/artists-db/metadata/90-MediaMetadata.p 	550194    297132    1592977   
/Volumes/Music/Discog/artists-db/metadata/91-MediaMetadata.p 	556189    300372    1610429   
/Volumes/Music/Discog/artists-db/metadata/92-MediaMetadata.p 	562331    303686    1628681   
/Volumes/Music/Discog/artists-db/metadata/93-MediaMetadata.p 	568279    306907    1646118   
/Volumes/Music/Discog/artists-db/metadata/94-MediaMetadata.p 	574228    310087    1663140   
/Volumes/Music/Discog/artists-db/metadata/95-MediaMetadata.p 	580266    313313    1680605   
/Volumes/Music/Discog/artists-db/metadata/96-MediaMetadata.p 	586223   

## Artist ID --> Genre, Style, Artists Lookup Table

In [5]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Dec 16, 2019 23:11:56 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p 	4253
/Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p 	8450
/Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p 	13293
/Volumes/Music/Discog/albums-db/metadata/11-ArtistMetadata.p 	17415
/Volumes/Music/Discog/albums-db/metadata/12-ArtistMetadata.p 	22696
/Volumes/Music/Discog/albums-db/metadata/13-ArtistMetadata.p 	26180
/Volumes/Music/Discog/albums-db/metadata/14-ArtistMetadata.p 	31942
/Volumes/Music/Discog/albums-db/metadata/15-ArtistMetadata.p 	37935
/Volumes/Music/Discog/albums-db/metadata/16-ArtistMetadata.p 	42771
/Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p 	46947
/Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p 	50218
/Volumes/Music/Discog/albums-db/metadata/19-ArtistMetadata.p 	56082
/Volumes/Music/Discog/albums-db/me

## Album ID --> Name, Ref, Artists Lookup Table

In [67]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Dec 15, 2019 17:51:45 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p 	20548
/Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p 	38649
/Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p 	61791
/Volumes/Music/Discog/albums-db/metadata/11-ArtistAlbums.p 	76305
/Volumes/Music/Discog/albums-db/metadata/12-ArtistAlbums.p 	91877
/Volumes/Music/Discog/albums-db/metadata/13-ArtistAlbums.p 	103117
/Volumes/Music/Discog/albums-db/metadata/14-ArtistAlbums.p 	125338
/Volumes/Music/Discog/albums-db/metadata/15-ArtistAlbums.p 	142259
/Volumes/Music/Discog/albums-db/metadata/16-ArtistAlbums.p 	159355
/Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p 	171733
/Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p 	187751
/Volumes/Music/Discog/albums-db/metadata/19-ArtistAlbums.p 	207043
/Volumes/Music/Discog/albums-db/metadata/2-Artist

# Master Lookup Tests

In [69]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Testing ArtistID --> Name
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 11.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
	Dim = 601757

Testing ArtistID --> Ref
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
	Dim = 601757

Testing ArtistID --> Variations
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
	Dim = 601757

Testing ArtistID --> Genre
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 9.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
	Dim = 447007

Testing ArtistID --> Style
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 13.7MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
	Dim = 447007

Testing ArtistID

# Pandas DB

## Slim Artist DB

In [24]:
from masterdb import getSlimArtistDB

In [25]:
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

Current Time is Sat Dec 14, 2019 17:41:14 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 11.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 13.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Creating Pandas DataFrame for 589990 Artists
	Shape --> (589990, 1)
  Finding Real Artist Name
	Shape --> (589990, 3)
  Removing None Artist
	Shape --> (589990, 3)
  Finding Disc Artist Name
	Shape --> (589990, 4)
DataFrame Shape is (589990, 4)
Current Time is Sat Dec 14, 2019 17:41:30 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 16 seconds.
Saving Master Artist DB File
  --> This file is 47.0MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist DB

In [29]:
from masterdb import getArtistDB

In [31]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

Current Time is Sat Dec 14, 2019 17:44:20 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 11.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 13.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToVariations.p
  --> This file is 22.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToVariations.p
Creating Pandas DataFrame for 589990 Artists
	Shape --> (589990, 1)
  Joining Ref
	Shape --> (589990, 2)
  Joining Variations
	Shape --> (589990, 3)
  Finding Real Artist Name
	Shape --> (589990, 6)
DataFrame Shape is (589990, 6)
Current Time is Sat Dec 14, 2019 17:44:53 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 32 seconds.
Saving Mas

,Name,Ref,Variations,Known,Artist,Num
1000,Dave Clarke,/artist/1000-Dave-Clarke,"[Dave Clarke, 7 Red 7, Clarck, Clarke, Clarke,...",True,Dave Clarke,NaN
1000500,Club Pulse,/artist/1000500-Club-Pulse,[Club Pulse],True,Club Pulse,NaN
100200,Dike,/artist/100200-Dike,"[Dike, D.I.K.E., Dike D, Uchegdu]",True,Dike,NaN
1002000,Larry Stokes,/artist/1002000-Larry-Stokes,[Larry Stokes],True,Larry Stokes,NaN
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez,[Gidd Sanchez],True,Gidd Sanchez,NaN


## Artist Metadata DB

In [34]:
from masterdb import getArtistMetadataDB

In [35]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

Current Time is Sat Dec 14, 2019 17:47:09 for 
=================================== Creating Artist Metadata DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 9.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
  --> This file is 30.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
Creating Pandas DataFrame for 442853 Artists
	Shape --> (442853, 1)
  Joining Style
	Shape --> (442853, 2)
  Joining Collaboration
	Shape --> (442853, 3)
DataFrame Shape is (442853, 3)
Current Time is Sat Dec 14, 2019 17:48:11 for Done with 
=================================== Creating Artist Metadata DB ===================================
Process [{0}] took 1.0 minutes.
Saving Master Artist Me

,Genre,Style,Collaboration
1000,"{'Electronic': 53, 'Hip Hop': 1, 'Non-Music': ...","{'Techno': 45, 'Electro': 16, 'Downtempo': 1, ...","{'Dave Clarke': 52, 'Bang The Future': 1, 'Mr...."
1000500,{'Electronic': 1},{'Progressive Trance': 1},{'Club Pulse': 1}
100200,{'Hip Hop': 11},"{'Conscious': 7, 'Pop Rap': 2, '': 3}","{'Dike': 11, 'ABS (2)': 2, 'Creutzfeld&Jakob':..."
1002000,{},{},{}
1005400,{'Rock': 5},"{'Alternative Rock': 1, 'Indie Rock': 1, 'Math...","{'Gapeseed': 5, 'Gerling': 1}"


## Artist Albums DB

In [36]:
def getArtistAlbumsDBxx(loadRefs=False):
    start, cmt = clock("\n===================================== Creating Artist Albums DB =====================================")
    from pandas import Series, DataFrame
    print("Loading ArtistID Data")
    artistIDtoAlbumNames  = Series(disc.getArtistIDToAlbumNamesData())
    if loadRefs:
        artistIDtoAlbumRefs   = Series(disc.getArtistIDToAlbumRefsData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoAlbumNames.shape[0]))
    cols = ["Albums"]
    discdf = DataFrame(artistIDtoAlbumNames)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)
    
    return discdf

In [37]:
from masterdb import getArtistAlbumsDB

In [39]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

Current Time is Sat Dec 14, 2019 17:50:16 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 168.0MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 589677 Artists
	Shape --> (589677, 1)
DataFrame Shape is (589677, 1)
Current Time is Sat Dec 14, 2019 17:51:53 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 1.6 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Albums DB

In [45]:
from masterdb import getAlbumDB

In [46]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

Current Time is Sat Dec 14, 2019 17:53:02 for 
=================================== Creating Artist Album DB ===================================
Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 31.9MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 44.8MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtists.p
  --> This file is 18.1MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtists.p
Creating Pandas DataFrame for 1372657 Albums
	Shape --> (1372657, 1)
  Joining Ref
	Shape --> (1372657, 2)
  Joining Artists
	Shape --> (1372657, 3)
DataFrame Shape is (1372657, 3)
Current Time is Sat Dec 14, 2019 17:54:02 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 59 seconds.


,Name,Ref,Artists
1012077,Live,/Dave-Clarke-Live/master/1012077,[1000]
1258261,Walls Of Genius And Miracle,/Walls-Of-Genius-and-Miracle-Walls-Of-Genius-A...,"[1009227, 1000, 182635]"
2546978,RA.EX007 Dave Clarke,/Dave-Clarke-RAEX007-Dave-Clarke/release/2546978,[1000]
44815,Archive One,/Dave-Clarke-Archive-One/master/44815,[1000]
44831,Devil's Advocate,/Dave-Clarke-Devils-Advocate/master/44831,[1000]


## Artist Album ID --> Known Albums

In [55]:
from masterdb import getArtistAlbumKnownDB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

Current Time is Sat Dec 14, 2019 18:29:20 for 
=================================== Creating Artist Album DB ===================================
Creating Pandas DataFrame for 589677 Arist Albums
	Shape --> (589677, 3)
DataFrame Shape is (589677, 3)
Current Time is Sat Dec 14, 2019 18:29:51 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 31 seconds.


,Known Albums,All Albums,Albums
1000,53,53,"{'44815': ['Archive One', 'Albums', True], '44..."
1000500,1,1,"{'1178875': ['Peak Controller', 'Miscellaneous..."
100200,9,9,"{'50511': ['PottpÃ¼ree', 'Albums', True], '219..."
1002000,3,3,"{'1002108': ['Are You Faithful?', 'Instruments..."
1002600,1,1,"{'1180905': ['Quiero', 'Vocals', True]}"


# Joining Artist ID DataFrame

In [61]:
from masterdb import createMasterDB

In [62]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

Current Time is Sat Dec 14, 2019 18:32:34 for 
=================================== Creating Artist ID DB ===================================
Creating Pandas DataFrame for 589990 Arist IDs
  Joining Artist Metadata
	Shape --> (589990, 9)
  Joining Artist Albums
	Shape --> (589990, 12)
Current Time is Sat Dec 14, 2019 18:32:35 for Done with 
=================================== Creating Artist ID DB ===================================
Process [{0}] took 902.0 millseconds.
Saving data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 543.5MB.
Saved data to /Volumes/Music/Discog/db/MasterDB.p
  --> This file is 543.5MB.
